In [1]:
import altair as alt
import pandas as pd
from altair.expr import datum, if_, format

In [3]:
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

In [ ]:
data1 = pd.read_csv('data/all_gen_wyear.csv', dtype={'EDRPOU': object, 'KOATUU': object})
df = pd.melt(data1, list(data1.columns[:5]) + list(data1.columns[11:]), list(data1.columns[5:11]))
data1.head()

In [ ]:
data2 = pd.read_csv('data/declarations_full.csv', sep=';')
# data2.head()

In [ ]:
base = alt.Chart(data1).properties(height=500, width=800, title = alt.TitleParams(text = 'Top 100 companies that produce biggest amounts of dangerous waste in Ukraine', 
                    subtitle = ['', 'The score of how dangerous produced waste is has been calculated by ПЗУВ formula', "Hover to get more detils on company's waste priduction", ''],
                    font = 'Ubuntu Mono', fontSize = 20, color = '#3E454F', subtitleFont = 'Ubuntu Mono',
                                      subtitleFontSize = 12, subtitleColor = '#3E454F', anchor='start'))


marks = base.mark_point(size=50).encode(
    x = alt.X('Відходів в одиницях Пзув:Q', scale=alt.Scale(type='log', base=2)),
    y = alt.Y('Згенеровано відходів в тоннах:Q', scale=alt.Scale(type='log', base=2)),
    tooltip = [alt.Tooltip('Name:N'), alt.Tooltip('Згенеровано відходів в тоннах:N'), alt.Tooltip('Відходів в одиницях Пзув:N'), alt.Tooltip('Відходи I класу:N'), alt.Tooltip('Відходи II класу:N') , alt.Tooltip('Відходи III класу:N') , alt.Tooltip('Відходи IV класу:N') ]
).transform_window(
    window=[{'op': 'rank', 'as': 'rank'}],
    sort=[{'field': 'Відходів в одиницях Пзув', 'order': 'descending'}]
).transform_filter('datum.rank <= 100')

marks.configure_view(strokeOpacity=0
).configure_axis(gridColor='#FFFFFF', ticks=False, domainColor='#FFFFFF', labelFont='ITC Franklin Gothic Demi', labelFontSize=12, labelColor='#635F5D', labelPadding=10, labelFontWeight='bold'
).configure(background = '#EFECEA', padding = 22)

In [ ]:
select_box = alt.binding_select(options=list(df['variable'].unique()))
selection = alt.selection_single(name='Y axis ', fields=['variable'], bind=select_box)
bar_selector = alt.selection_single(fields=['EDRPOU'], on='mouseover')


base = alt.Chart(df).properties(height=500, width=800, title = alt.TitleParams(text = '10 Highest Waste Generators in Ukraine', 
                    subtitle = ['', 'Top companies that generate the biggest amount of waste by categories', 'From Class I - highly dangerous waste, to Class IV - safe waste',
                               'ПЗУВ(Показник Загального Утворення Відходів) - index of how dangerous and quantity of produced waste', 'Hover to get more detils', ''],
                    font = 'Ubuntu Mono', fontSize = 20, color = '#3E454F', subtitleFont = 'Ubuntu Mono',
                                      subtitleFontSize = 12, subtitleColor = '#3E454F', anchor='start'))

bar_ch = base.mark_bar().encode(
    x = alt.X('EDRPOU:N', axis=alt.Axis(title="Companies(ЄДРПОУ)", labelAngle=0, titlePadding=42, titleFontSize=12), sort='-y'),
    y = alt.Y('value:Q', axis=alt.Axis(title='', titlePadding=42, titleFontSize=12, titleAngle=90)), 
    opacity=alt.condition(bar_selector, alt.value(1.0), alt.value(0.6)),
    tooltip = [alt.Tooltip('Name:N'), alt.Tooltip('Address:N'), alt.Tooltip('Contacts:N'), alt.Tooltip('KOATUU:N') ]
#     sort=alt.SortField(field='Згенеровано відходів в тоннах:Q', order='descending'))
).add_selection(
    selection, bar_selector
).transform_filter(
    selection
).transform_window(
    window=[{'op': 'rank', 'as': 'rank'}],
    sort=[{'field': 'value', 'order': 'descending'}]
).transform_filter('datum.rank <= 10')


(bar_ch).configure_view(strokeOpacity=0
).configure_axis(gridColor='#FFFFFF', ticks=False, domainColor='#FFFFFF', labelFont='ITC Franklin Gothic Demi', labelFontSize=12, labelColor='#635F5D', labelPadding=10, labelFontWeight='bold'
).configure(background = '#EFECEA', padding = 22)